In [30]:
import getpass
import os
import json
from langchain_neo4j import Neo4jGraph
from langchain_neo4j import GraphCypherQAChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [31]:
# Load .env file
load_dotenv(override=True)

# Read environment variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORDD = os.getenv("NEO4J_PASSWORDD")
NEO4J_DATABASE1 = os.getenv("NEO4J_DATABASE1")
NEO4J_DATABASE2 = os.getenv("NEO4J_DATABASE2")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

In [32]:
print(NEO4J_USERNAME, NEO4J_PASSWORDD, NEO4J_URL, NEO4J_DATABASE1)

neo4j speartg123 bolt://127.0.0.1:7687 neo4j


##### **GraphRAG For Calls Transcript**

In [18]:
callsgraph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password="speartg123",
    database=NEO4J_DATABASE1,
    enhanced_schema=True
)

In [19]:
callsgraph.refresh_schema()
print(callsgraph.schema)

Node properties:
- **IdealTargetCustomer**
  - `personal_interests`: STRING Available options: ['[     { label: "golf", url: "https://www.pgatour.c', '[     { label: "triathlon", url: "https://www.tria', '[     { label: "mountain biking", url: "https://ww', '[     { label: "classic cars", url: "https://www.h', '[     { label: "art house cinema", url: "https://w']
  - `meta_url`: STRING Available options: ['']
  - `objection_templates`: STRING Available options: ['[     { type: "cost", response: "ROI-focused prici', '[     { type: "compliance", response: "Regular thi', '[     { type: "integration", response: "private se', '[     { type: "downtime", response: "Rapid deploym', '[     { type: "cost", response: "Flexible licensin']
  - `company_size`: STRING Available options: ['500+ employees', '1000+ beds', '50-300 employees', '1000-5000 employees', '50-10,000 employees']
  - `location`: STRING Available options: ['Major US Urban Centers', 'Nationwide urban & suburban hospitals', 'Global 

**SIMPLEST**

In [20]:
callschain = GraphCypherQAChain.from_llm(
    graph=callsgraph, llm=llm,
    verbose=True, allow_dangerous_requests=True
)

response = callschain.invoke({"query": "give me complete call records for last 2 calls"})

print(response['result'])



> Entering new GraphCypherQAChain chain...


Generated Cypher:
MATCH (cs:CallSession)
WITH cs
ORDER BY cs.session_id DESC
LIMIT 2
OPTIONAL MATCH path=(cs)-[*0..10]-(n)
RETURN cs, nodes(path) AS nodes, relationships(path) AS relationships
ORDER BY cs.session_id DESC;


KeyboardInterrupt: 

**BETTER**

In [ ]:
# 1. Define the custom prompt to get better, more natural answers.
QA_PROMPT_TEMPLATE = """You are an assistant that helps to form nice and human readable answers.
The information part contains the provided information that you must use to construct an answer.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound like a response to the question. Do not mention that you are using the information from the graph.

Information:
{context}

Question: {question}
Helpful Answer:"""

QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=QA_PROMPT_TEMPLATE,
)

# 2. Create the chain using the correct class from the langchain-neo4j package.
callschain = GraphCypherQAChain.from_llm(
    graph=callsgraph,
    llm=llm,
    qa_prompt=QA_PROMPT, # Pass our custom prompt here
    verbose=True,
    allow_dangerous_requests=True
)

# 3. Invoke the chain. It will now work as expected.
response = callschain.invoke({"query": "give me complete call records for last 2 calls"})

print(response['result'])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:CallSession)
WITH c
ORDER BY c.session_id DESC
LIMIT 2
OPTIONAL MATCH (c)-[:FOCUSES_ON]->(prod:Product)
OPTIONAL MATCH (person:Person)-[:PARTICIPATED_IN]->(c)
OPTIONAL MATCH (person)-[:MADE_BY]->(u)
OPTIONAL MATCH (person)-[:MATCHES_PROFILE]->(itc:IdealTargetCustomer)
OPTIONAL MATCH (raisedNode)-[:RAISED_IN]->(c)
OPTIONAL MATCH (u)-[nr:NEXT]->(v)
WHERE EXISTS((u)-[:RAISED_IN]->(c)) AND EXISTS((v)-[:RAISED_IN]->(c))
RETURN c, prod, collect(DISTINCT person) AS participants, collect(DISTINCT u) AS utterances, collect(DISTINCT nr) AS next_relationships, collect(DISTINCT v) AS next_nodes, collect(DISTINCT itc) AS matched_profiles, collect(DISTINCT raisedNode) AS raised_nodes
ORDER BY c.session_id DESC;
Full Context:
[{'c': {'quality_status': 'High-confidence', 'matched_icp_segment': 'Retail-Enterprise', 'quality_score': 62, 'session_id': 'call_transcript_4', 'product_focus': 'Georgia Senate Bill 68 and 69 compliance sol

**MORE STRUCTURED: CYPHER + ANALYST OUTPUT**

In [ ]:
callsgraph.schema

'Node properties:\n- **CallSession**\n  - `session_id`: STRING Available options: [\'call_transcript_1\', \'call_transcript_2\', \'call_transcript_3\', \'call_transcript_4\']\n  - `product_focus`: STRING Available options: [\'Georgia Senate Bill 68 compliance solution\', \'Georgia Senate Bill 68 and 69 compliance solution\']\n  - `outcome`: STRING Available options: [\'Meeting Scheduled\']\n  - `quality_score`: INTEGER Min: 62, Max: 101\n  - `quality_status`: STRING Available options: [\'High-confidence\']\n  - `matched_icp_segment`: STRING Available options: [\'Retail-Enterprise\']\n- **Product**\n  - `name`: STRING Available options: [\'Georgia Senate Bill 68 compliance solution\', \'Georgia Senate Bill 68 and 69 compliance solution\']\n- **Person**\n  - `name`: STRING Available options: [\'Arison\', \'Unnamed Gatekeeper\', \'Dale Spear\', \'Eric Spear\', \'Ted\']\n  - `role`: STRING Available options: [\'Agent\', \'Gatekeeper\', \'Recipient\']\n- **GatekeeperDialogue**\n  - `text`: 

In [ ]:
# NEW: Create a more advanced prompt for Cypher Generation
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Cypher translator who understands the graph schema and writes Cypher queries to answer questions.
Convert the user's question into a Cypher query based on the schema below.
Pay attention to the entities and relationships mentioned in the question and map them to the schema.

Node properties:
CallSession {{session_id: STRING, product_focus: STRING, outcome: STRING, quality_score: INTEGER, quality_status: STRING, matched_icp_segment: STRING}}
Product {{name: STRING}}
Person {{name: STRING, role: STRING}}
GatekeeperDialogue {{text: STRING, turn_number: INTEGER}}
Opening {{text: STRING, turn_number: INTEGER}}
Closing {{text: STRING, turn_number: INTEGER}}
CustomerBuyingSignal {{text: STRING, turn_number: INTEGER}}
AgentQuestion {{text: STRING, turn_number: INTEGER}}
AgentResponse {{text: STRING, turn_number: INTEGER}}
CustomerQuestion {{text: STRING, turn_number: INTEGER}}
CustomerPainPoint {{text: STRING, turn_number: INTEGER}}
IdealTargetCustomer {{affinity_language_style: STRING, age_range: STRING, blog_url: STRING, company_size: STRING, completed_call_count: INTEGER, facebook_url: STRING, job_titles: LIST, linkedin_url: STRING, location: STRING, meeting_conversion_rate: FLOAT, meta_url: STRING, objection_templates: STRING, personal_interests: STRING, preferences: LIST, professional_interests: STRING, purchase_frequency: STRING, reddit_url: STRING, segment: STRING, substack_url: STRING, top_pain_points: LIST, twitter_url: STRING, x_url: STRING, description: STRING}}
CustomerResponse {{text: STRING, turn_number: INTEGER}}
RapportBuilding {{text: STRING, turn_number: INTEGER}}
TechnicalIssue {{text: STRING, turn_number: INTEGER}}

Relationship properties:

The relationships:
(:CallSession)-[:FOCUSES_ON]->(:Product)
(:Person)-[:PARTICIPATED_IN]->(:CallSession)
(:Person)-[:MADE_BY]->(:GatekeeperDialogue)
(:Person)-[:MADE_BY]->(:Opening)
(:Person)-[:MADE_BY]->(:AgentQuestion)
(:Person)-[:MADE_BY]->(:Closing)
(:Person)-[:MADE_BY]->(:AgentResponse)
(:Person)-[:MADE_BY]->(:RapportBuilding)
(:Person)-[:MADE_BY]->(:TechnicalIssue)
(:Person)-[:MADE_BY]->(:CustomerResponse)
(:Person)-[:MADE_BY]->(:CustomerBuyingSignal)
(:Person)-[:MADE_BY]->(:CustomerPainPoint)
(:Person)-[:MADE_BY]->(:CustomerQuestion)
(:Person)-[:MATCHES_PROFILE]->(:IdealTargetCustomer)
(:GatekeeperDialogue)-[:NEXT]->(:GatekeeperDialogue)
(:GatekeeperDialogue)-[:NEXT]->(:CustomerResponse)
(:GatekeeperDialogue)-[:RAISED_IN]->(:CallSession)
(:Opening)-[:NEXT]->(:CustomerBuyingSignal)
(:Opening)-[:NEXT]->(:CustomerResponse)
(:Opening)-[:RAISED_IN]->(:CallSession)
(:Closing)-[:NEXT]->(:CustomerResponse)
(:Closing)-[:RAISED_IN]->(:CallSession)
(:CustomerBuyingSignal)-[:NEXT]->(:AgentQuestion)
(:CustomerBuyingSignal)-[:RAISED_IN]->(:CallSession)
(:AgentQuestion)-[:NEXT]->(:CustomerBuyingSignal)
(:AgentQuestion)-[:NEXT]->(:CustomerResponse)
(:AgentQuestion)-[:NEXT]->(:CustomerPainPoint)
(:AgentQuestion)-[:NEXT]->(:CustomerQuestion)
(:AgentQuestion)-[:RAISED_IN]->(:CallSession)
(:AgentResponse)-[:NEXT]->(:CustomerResponse)
(:AgentResponse)-[:RAISED_IN]->(:CallSession)
(:CustomerQuestion)-[:NEXT]->(:AgentResponse)
(:CustomerQuestion)-[:NEXT]->(:RapportBuilding)
(:CustomerQuestion)-[:RAISED_IN]->(:CallSession)
(:CustomerPainPoint)-[:NEXT]->(:AgentQuestion)
(:CustomerPainPoint)-[:RAISED_IN]->(:CallSession)
(:CustomerResponse)-[:NEXT]->(:Opening)
(:CustomerResponse)-[:NEXT]->(:AgentQuestion)
(:CustomerResponse)-[:NEXT]->(:Closing)
(:CustomerResponse)-[:NEXT]->(:RapportBuilding)
(:CustomerResponse)-[:NEXT]->(:TechnicalIssue)
(:CustomerResponse)-[:RAISED_IN]->(:CallSession)
(:RapportBuilding)-[:NEXT]->(:CustomerResponse)
(:RapportBuilding)-[:RAISED_IN]->(:CallSession)
(:TechnicalIssue)-[:NEXT]->(:TechnicalIssue)
(:TechnicalIssue)-[:NEXT]->(:AgentQuestion)
(:TechnicalIssue)-[:RAISED_IN]->(:CallSession)

Schema: {schema}
Question: {question}
Cypher Query:
"""

CYPHER_PROMPT = PromptTemplate(
    input_variables=["schema", "question"],
    template=CYPHER_GENERATION_TEMPLATE
)

# NEW: An "analyst" prompt for the final QA step
ANALYST_QA_PROMPT_TEMPLATE = """You are a conversation analyst and your goal is to help improve call center scripts.
You are given a question and a set of data retrieved from a call transcript graph database.
Your task is to analyze this data to identify patterns and provide actionable insights.

Follow these steps:
1.  Briefly summarize the direct answer to the question based on the provided data.
2.  Identify patterns or common themes among the successful examples. What makes them work? (e.g., tone, structure, key phrases).
3.  Provide 2-3 concrete, actionable recommendations for improving the calling agent's script based on your analysis.
4.  Present your answer in a clear, structured format using markdown.

Data from the graph:
{context}

Original Question: {question}
Your Expert Analysis and Recommendations:"""

ANALYST_QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=ANALYST_QA_PROMPT_TEMPLATE,
)


# Create the chain using the new analyst prompts
callschain = GraphCypherQAChain.from_llm(
    graph=callsgraph,
    llm=llm,
    cypher_prompt=CYPHER_PROMPT,  # Use the new Cypher prompt
    qa_prompt=ANALYST_QA_PROMPT, # Use the new Analyst QA prompt
    verbose=True,
    allow_dangerous_requests=True
)

# Invoke the chain with the same query
response = callschain.invoke({"query": "give me summary of call records for last 2 calls"})

print(response['result'])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (cs:CallSession)
WITH cs
ORDER BY cs.session_id DESC
LIMIT 2
CALL {
  WITH cs
  OPTIONAL MATCH (cs)-[:FOCUSES_ON]->(prod:Product)
  RETURN prod.name AS product_name
}
CALL {
  WITH cs
  OPTIONAL MATCH (person:Person)-[:PARTICIPATED_IN]->(cs)
  RETURN collect(distinct person.name + ' (' + person.role + ')') AS participants
}
CALL {
  WITH cs
  OPTIONAL MATCH (o:Opening)-[:RAISED_IN]->(cs)
  RETURN collect(distinct o.text)[0..3] AS opening_examples, size(collect(distinct o)) AS opening_count
}
CALL {
  WITH cs
  OPTIONAL MATCH (cl:Closing)-[:RAISED_IN]->(cs)
  RETURN collect(distinct cl.text)[0..3] AS closing_examples, size(collect(distinct cl)) AS closing_count
}
CALL {
  WITH cs
  OPTIONAL MATCH (aq:AgentQuestion)-[:RAISED_IN]->(cs)
  RETURN size(collect(distinct aq)) AS agent_question_count
}
CALL {
  WITH cs
  OPTIONAL MATCH (cr:CustomerResponse)-[:RAISED_IN]->(cs)
  RETURN size(collect(distinct cr)) AS customer_res

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (cs) { ... }} {position: line: 5, column: 1, offset: 67} for query: "MATCH (cs:CallSession)\nWITH cs\nORDER BY cs.session_id DESC\nLIMIT 2\nCALL {\n  WITH cs\n  OPTIONAL MATCH (cs)-[:FOCUSES_ON]->(prod:Product)\n  RETURN prod.name AS product_name\n}\nCALL {\n  WITH cs\n  OPTIONAL MATCH (person:Person)-[:PARTICIPATED_IN]->(cs)\n  RETURN collect(distinct person.name + ' (' + person.role + ')') AS participants\n}\nCALL {\n  WITH cs\n  OPTIONAL MATCH (o:Opening)-[:RAISED_IN]->(cs)\n  RETURN collect(distinct o.text)[0..3] AS opening_examples, size(collect(distinct o)) AS opening_count\n}\nCALL {\n  WITH cs\n  OPTIONAL MATCH (cl:Closing)-[:RAISED_IN]->(cs)\n  RETURN collect(distinct

Full Context:
[{'session_id': 'call_transcript_4', 'product_focus': 'Georgia Senate Bill 68 and 69 compliance solution', 'product_name': 'Georgia Senate Bill 68 and 69 compliance solution', 'outcome': 'Meeting Scheduled', 'quality_score': 62, 'quality_status': 'High-confidence', 'matched_icp_segment': 'Retail-Enterprise', 'participants': ['Arison (Agent)', 'Ted (Recipient)'], 'opening_count': 1, 'opening_examples': ["Hi, Ted. I was hoping you could help me. I need to speak with the person responsible for compliance regarding Georgia's Senate Bill 68 and 69. Can you put me through?"], 'closing_count': 0, 'closing_examples': [], 'agent_question_count': 7, 'customer_response_count': 5, 'pain_point_count': 1, 'pain_points': ["Probably result in a lawsuit if that was the case. There's a long pause in between our conversation and something What's going on?"]}, {'session_id': 'call_transcript_3', 'product_focus': 'Georgia Senate Bill 68 compliance solution', 'product_name': 'Georgia Senate Bi

##### **GraphRAG For Reference PDFs**

In [ ]:
pdfgraph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORDD,
    database=NEO4J_DATABASE2,
    enhanced_schema=True
)

In [ ]:
pdfgraph.refresh_schema()
print(pdfgraph.schema)

Node properties:
- **Document**
  - `id`: STRING Example: "9b2e3f806199be76c6b831ad6b4639be"
  - `text`: STRING Available options: ['Alen  Sultanic    FF  Swipe  File  Part  1       C', 'it   becomes   a   shortcut   to   ___________    ', 'And  it  has  nothing  to  do  with  your  _______', 'All   that   and   more   will   happen   for   yo', 'you   know   how   to   ________________   …     Y', '______________     And   Im   not   the   only   o', 'TIMELINE  LANGUAGE   First  you’ll___,  then  once', 'So   after   you   know   how   to   ______,   ___', 'ASSUMPTIVE  QUESTIONS   They  always  open  with  ', "OPENERS    The  Two  Types  of  Coaches   There's "]
  - `creator`: STRING Available options: ['PyPDF']
  - `creationdate`: STRING Available options: ['']
  - `producer`: STRING Available options: ['Skia/PDF m134 Google Docs Renderer']
  - `source`: STRING Available options: ['pdfs/1. Copywriting Swipe Files (Alen).pdf']
  - `total_pages`: INTEGER Min: 83, Max: 83
  - `page`: IN

In [ ]:
pdfchain = GraphCypherQAChain.from_llm(
    graph=pdfgraph, llm=llm, verbose=True, allow_dangerous_requests=True
)
response = pdfchain.invoke({"query": "it has nothing to do with what?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Document)-[:MENTIONS]->(t:Thing)
WHERE d.text =~ '(?i).*has\\s+nothing\\s+to\\s+do\\s+with.*'
RETURN DISTINCT t.id AS id;
Full Context:
[]

> Finished chain.


{'query': 'it has nothing to do with what?',
 'result': "I don't know the answer."}